In [191]:
import torch
import torch.nn.functional as F
import matplotlib as plt
%matplotlib inline

In [192]:
words = open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [193]:
len(words)

32033

In [194]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [195]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
    
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [196]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [197]:
X[:6,]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0]])

In [199]:
Y[:6]

tensor([ 5, 13, 13,  1,  0, 15])

In [200]:
g = torch.Generator().manual_seed(2147483647)
# W = torch.randn((27, 27), generator=g, requires_grad=True)
C = torch.randn((27, 2), generator=g)

In [201]:
C[:6,]

tensor([[ 1.5674, -0.2373],
        [-0.0274, -1.1008],
        [ 0.2859, -0.0296],
        [-1.5471,  0.6049],
        [ 0.0791,  0.9046],
        [-0.4713,  0.7868]])

In [202]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [203]:
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)

In [204]:
# we would like to do something like this
emb @ W1 + b1
# but emb is the wrong shape (3D)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [205]:
# we want emb to be 32 by 6, the six being 2 by 3 first layer neurons
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]], 1).shape

torch.Size([32, 6])

In [206]:
# to cope with any block_size
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [207]:
# more efficiently, as all torch tensors are stored as one-dimensional
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [209]:
xDim0 = X.shape[0]
h = torch.zeros((xDim0, 100))
for i in range(xDim0):
    h[i] = emb.view(-1, 6)[i,:] @ W1 + b1
#     print(h[i])

In [210]:
h = torch.zeros((xDim0, 100))
for i in range(xDim0):
    h[i] = torch.tanh(emb.view(-1, 6)[i,:] @ W1 + b1)
#     print(h[i])

In [211]:
print(h.shape, h.dtype)
h

torch.Size([32, 100]) torch.float32


tensor([[-0.9348,  1.0000,  0.9258,  ...,  0.9786, -0.1926,  0.9515],
        [ 0.2797,  0.9997,  0.7675,  ...,  0.9929,  0.9992,  0.9981],
        [-0.9960,  1.0000, -0.8694,  ..., -0.5159, -1.0000, -0.0069],
        ...,
        [-0.9996,  1.0000, -0.9273,  ..., -0.9999, -0.9974, -0.9970],
        [-0.9043,  1.0000,  0.9868,  ..., -0.7859, -0.4819,  0.9981],
        [-0.9048,  1.0000,  0.9553,  ...,  0.9866,  1.0000,  0.9907]])

In [212]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)
print(W2.shape, W2.dtype)
W2

torch.Size([100, 27]) torch.float32


tensor([[ 2.7139e+00,  1.7004e+00, -5.2536e-01,  ...,  2.2673e-01,
          5.7200e-01,  1.2515e+00],
        [ 3.9931e-01,  5.8364e-01, -1.8971e+00,  ..., -6.0466e-01,
          9.6749e-01, -4.2308e-01],
        [-2.5798e-01, -3.0600e-01, -6.5714e-01,  ..., -2.0904e+00,
         -2.6559e-01,  1.1214e+00],
        ...,
        [-9.5891e-01, -4.5719e-04, -2.0679e-01,  ...,  2.1099e+00,
          1.3437e-01,  2.0550e+00],
        [-1.6093e+00, -6.5691e-02, -2.3609e+00,  ..., -2.4089e-01,
         -2.8725e-01,  4.5899e-02],
        [-1.2685e+00, -1.3525e+00, -1.8254e-01,  ..., -2.0127e+00,
          1.6496e-02, -1.2098e+00]])

In [213]:
logits = h[:,:29] @ W2[:29,:]
logits = h @ W2 + b2

In [226]:
print(logits.shape, logits.dtype)
logits

torch.Size([32, 27]) torch.float32


tensor([[-9.6305e+00,  1.4939e+01,  1.2592e+01,  8.6687e+00, -1.5139e+01,
          3.2233e+00,  7.3641e+00, -7.0430e+00, -5.1357e+00,  2.2856e-02,
          2.0298e+00,  9.3355e+00, -8.6624e-01,  6.0250e+00, -9.1530e+00,
          8.2638e-01, -3.4893e-01, -1.7576e-01, -8.5076e+00, -3.7636e+00,
         -2.3550e+00,  1.5538e+01,  8.6370e+00,  2.4770e+00, -4.3730e+00,
         -7.8310e-01,  3.4189e+00],
        [-1.0677e+01,  9.7100e+00,  2.9911e+00,  3.4321e+00, -7.5804e+00,
          3.9931e+00,  7.9634e-01,  3.1430e+00, -7.0838e-01,  1.6692e+00,
         -7.3368e+00,  1.1808e+01,  5.5312e+00,  2.7846e+00, -1.2023e+01,
          7.3979e+00, -1.5661e+00, -7.3063e+00, -1.0520e+01,  1.0278e-01,
         -2.3276e+00,  1.0422e+01,  3.3022e+00,  4.4635e+00, -1.2461e+01,
         -6.3743e+00, -3.6290e+00],
        [ 3.8093e+00,  8.2363e+00,  1.3480e+01,  1.2749e+01, -3.3486e+00,
         -4.9482e+00,  1.2121e+01, -1.0926e+01, -8.3533e+00,  4.6336e-01,
          1.3925e+00,  9.9100e+00,  3.75

In [219]:
counts = logits.exp()

In [227]:
prob = counts / counts.sum(1, keepdims=True)

In [230]:
print(prob.shape)
prob[0,:].sum().item()

torch.Size([32, 27])


1.0000001192092896

In [233]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(14.5864)

In [234]:
# ----------------- now made respectable :) -------------------

In [255]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
# for w in words[:5]:
for w in words:
    
#     print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
#         print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [256]:
X.shape, Y.shape # dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [257]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = (C, W1, b1, W2, b2)
for p in parameters:
    p.requires_grad = True

In [258]:
sum(p.nelement() for p in parameters)

3481

In [ ]:
for _ in range(10):
    # forward pass
    emb = C[X] # (32, 3, 2)
    XDim0 = X.shape[0]
    h = torch.zeros((XDim0, 100))
    for i in range(XDim0):
        h[i] = torch.tanh(emb.view(-1, 6)[i,:] @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad
# print(loss.item())

19.505229949951172
